# Data Acquisition, Cleaning, and Integration
### Project: Mapping and Modeling Drug-Related Deaths in Cook County

This notebook demonstrates interim progress on data acquisition, cleaning, and integration. There is secondary notebook that will conduct EDA.  
It documents the steps taken so far to prepare datasets for spatial and demographic analysis, as outlined in our Project Plan.


In [28]:
# imports

import pandas as pd
import requests
import hashlib
import os

from pathlib import Path

from textblob import TextBlob


In [2]:
# Move working directory to project root
project_root = Path(".").resolve().parent 
os.chdir(project_root)

print("Current working directory:", os.getcwd())

Current working directory: /Users/aaliyahali/Desktop/is477/IS477-FinalProject


In [3]:
# scripts/acquire_data.py

df_me = pd.read_csv("data/raw/Medical_Examiner_Case_Archive_20251104.csv", low_memory=False)

In [4]:
os.makedirs("data/raw", exist_ok=True)

# ACS 5-Year 2023 (data collected 2019–2023)
url = "https://api.census.gov/data/2023/acs/acs5"

variables = [
    # Population + race
    "B01003_001E",  # total population
    "B03002_003E",  # white (non-Hispanic)
    "B03002_004E",  # black
    "B03002_006E",  # asian
    "B03002_012E",  # hispanic

    # Income & poverty
    "B19013_001E",  # median income
    "B17001_002E",  # below poverty

    # Education
    "B15003_022E",  # bachelor's degree or higher

    # Age structure (male + female counts)
    "B01001_003E", "B01001_027E",  # under 18
    "B01001_007E", "B01001_031E",  # 18–34
    "B01001_010E", "B01001_034E",  # 35–64
    "B01001_020E", "B01001_044E",  # 65+

    # New: Unemployment, Health Insurance, Rent
    "B23025_002E",  # labor force
    "B23025_005E",  # unemployed
    "B27010_017E",  # no health insurance (all people)
    "B25064_001E"   # median gross rent
]

params = {
    "get": ",".join(["NAME"] + variables),
    "for": "tract:*",
    "in": "county:031 state:17"  # Cook County, IL
}

r = requests.get(url, params=params)
r.raise_for_status()

data = r.json()
cols = data[0]
rows = data[1:]
df_census = pd.DataFrame(rows, columns=cols)

# Rename columns for clarity
df_census = df_census.rename(columns={
    "B01003_001E": "TotalPop",
    "B03002_003E": "White_NonHisp",
    "B03002_004E": "Black",
    "B03002_006E": "Asian",
    "B03002_012E": "Hispanic",
    "B19013_001E": "MedianIncome",
    "B17001_002E": "BelowPoverty",
    "B15003_022E": "BachelorsOrHigher",
    "B01001_003E": "Male_Under18",
    "B01001_027E": "Female_Under18",
    "B01001_007E": "Male_18_34",
    "B01001_031E": "Female_18_34",
    "B01001_010E": "Male_35_64",
    "B01001_034E": "Female_35_64",
    "B01001_020E": "Male_65plus",
    "B01001_044E": "Female_65plus",
    "B23025_002E": "LaborForce",
    "B23025_005E": "Unemployed",
    "B27010_017E": "NoHealthInsurance",
    "B25064_001E": "MedianRent",
    "state": "STATE",
    "county": "COUNTY",
    "tract": "TRACT"
})

# Unique tract ID for merging
df_census["TRACT_FIPS"] = df_census["STATE"] + df_census["COUNTY"] + df_census["TRACT"].str.zfill(6)

df_census.to_csv("data/raw/census_tract_data.csv", index=False)
print(f"Saved enriched Census data for {len(df_census)} tracts.")


Saved enriched Census data for 1332 tracts.


In [5]:
df_census.head()

,NAME,TotalPop,White_NonHisp,Black,Asian,Hispanic,MedianIncome,BelowPoverty,BachelorsOrHigher,Male_Under18,...,Male_65plus,Female_65plus,LaborForce,Unemployed,NoHealthInsurance,MedianRent,STATE,COUNTY,TRACT,TRACT_FIPS
0,Census Tract 101; Cook County; Illinois,3726,1297,1376,137,809,69460,508,775,88,...,108,56,2579,260,130,1252,17,031,010100,17031010100
1,Census Tract 102.01; Cook County; Illinois,7588,1406,2301,376,2622,49639,1892,684,455,...,143,119,4007,317,40,1333,17,031,010201,17031010201
2,Census Tract 102.02; Cook County; Illinois,2609,967,949,212,423,55119,524,656,9,...,24,31,1449,135,36,1292,17,031,010202,17031010202
3,Census Tract 103; Cook County; Illinois,6311,3094,1298,342,1426,65871,431,1585,148,...,26,84,3923,166,0,1257,17,031,010300,17031010300
4,Census Tract 104; Cook County; Illinois,4282,3173,296,324,340,49017,463,999,25,...,83,18,2282,78,16,1248,17,031,010400,17031010400


In [6]:
df_me.head()

,Case Number,Date of Incident,Date of Death,Age,Gender,Race,Latino,Manner of Death,Primary Cause,Primary Cause Line A,...,Incident Zip Code,longitude,latitude,location,Residence City,Residence Zip,OBJECTID,Chicago Ward,Chicago Community Area,COVID Related
0,ME2025-05835,11/04/2025 02:09:00 PM,11/04/2025 01:57:00 PM,57.0,Male,Black,False,NaN,NaN,NaN,...,60621,-87.647293,41.779635,"(41.77963481, -87.64729284)",Chicago,60621,92724,16.0,ENGLEWOOD,False
1,ME2025-05834,09/24/2025 01:48:00 PM,11/04/2025 12:20:00 PM,55.0,Female,White,True,NaN,NaN,NaN,...,60609,-87.662232,41.806638,"(41.806638, -87.662232)",Chicago,60609,92725,20.0,NEW CITY,False
2,ME2025-05833,11/04/2025 12:00:00 AM,11/04/2025 12:40:00 PM,69.0,Female,White,True,NaN,NaN,NaN,...,60622,-87.696022,41.906421,"(41.906421, -87.6960225)",Chicago,60612,92720,26.0,WEST TOWN,False
3,ME2025-05832,11/04/2025 01:12:00 PM,11/04/2025 12:40:00 PM,75.0,Male,White,False,NaN,NaN,NaN,...,60622,-87.696022,41.906421,"(41.906421, -87.6960225)",Chicago,60622,92719,26.0,WEST TOWN,False
4,ME2025-05831,11/04/2025 01:41:00 PM,11/04/2025 09:48:00 AM,30.0,Male,Black,False,NaN,NaN,NaN,...,60429,-87.659707,41.581080,"(41.58108, -87.6597075)",Hazel Crest,60429,92722,NaN,NaN,False


### Exploratory Data Analysis 

In [7]:
len(df_me)

92642

In [8]:
df_me.dtypes

Case Number                object
Date of Incident           object
Date of Death              object
Age                       float64
Gender                     object
Race                       object
Latino                       bool
Manner of Death            object
Primary Cause              object
Primary Cause Line A       object
Primary Cause Line B       object
Primary Cause Line C       object
Secondary Cause            object
Gun Related                object
Opioid Related             object
Cold Related                 bool
Heat Related                 bool
Commissioner District     float64
Incident Address           object
Incident City              object
Incident Zip Code          object
longitude                 float64
latitude                  float64
location                   object
Residence City             object
Residence Zip              object
OBJECTID                    int64
Chicago Ward              float64
Chicago Community Area     object
COVID Related 

In [9]:
df_me.columns

Index(['Case Number', 'Date of Incident', 'Date of Death', 'Age', 'Gender',
       'Race', 'Latino', 'Manner of Death', 'Primary Cause',
       'Primary Cause Line A', 'Primary Cause Line B', 'Primary Cause Line C',
       'Secondary Cause', 'Gun Related', 'Opioid Related', 'Cold Related',
       'Heat Related', 'Commissioner District', 'Incident Address',
       'Incident City', 'Incident Zip Code', 'longitude', 'latitude',
       'location', 'Residence City', 'Residence Zip', 'OBJECTID',
       'Chicago Ward', 'Chicago Community Area', 'COVID Related'],
      dtype='object')

## Cleaning + Pre-Processing

### Census Data

In [10]:
df_census.isna().sum()
# no missing values

NAME                 0
TotalPop             0
White_NonHisp        0
Black                0
Asian                0
Hispanic             0
MedianIncome         0
BelowPoverty         0
BachelorsOrHigher    0
Male_Under18         0
Female_Under18       0
Male_18_34           0
Female_18_34         0
Male_35_64           0
Female_35_64         0
Male_65plus          0
Female_65plus        0
LaborForce           0
Unemployed           0
NoHealthInsurance    0
MedianRent           0
STATE                0
COUNTY               0
TRACT                0
TRACT_FIPS           0
dtype: int64

In [11]:
df_census.head()

,NAME,TotalPop,White_NonHisp,Black,Asian,Hispanic,MedianIncome,BelowPoverty,BachelorsOrHigher,Male_Under18,...,Male_65plus,Female_65plus,LaborForce,Unemployed,NoHealthInsurance,MedianRent,STATE,COUNTY,TRACT,TRACT_FIPS
0,Census Tract 101; Cook County; Illinois,3726,1297,1376,137,809,69460,508,775,88,...,108,56,2579,260,130,1252,17,031,010100,17031010100
1,Census Tract 102.01; Cook County; Illinois,7588,1406,2301,376,2622,49639,1892,684,455,...,143,119,4007,317,40,1333,17,031,010201,17031010201
2,Census Tract 102.02; Cook County; Illinois,2609,967,949,212,423,55119,524,656,9,...,24,31,1449,135,36,1292,17,031,010202,17031010202
3,Census Tract 103; Cook County; Illinois,6311,3094,1298,342,1426,65871,431,1585,148,...,26,84,3923,166,0,1257,17,031,010300,17031010300
4,Census Tract 104; Cook County; Illinois,4282,3173,296,324,340,49017,463,999,25,...,83,18,2282,78,16,1248,17,031,010400,17031010400


In [12]:
df_census.dtypes

NAME                 object
TotalPop             object
White_NonHisp        object
Black                object
Asian                object
Hispanic             object
MedianIncome         object
BelowPoverty         object
BachelorsOrHigher    object
Male_Under18         object
Female_Under18       object
Male_18_34           object
Female_18_34         object
Male_35_64           object
Female_35_64         object
Male_65plus          object
Female_65plus        object
LaborForce           object
Unemployed           object
NoHealthInsurance    object
MedianRent           object
STATE                object
COUNTY               object
TRACT                object
TRACT_FIPS           object
dtype: object

In [13]:
# convert string columns to numeric values for computational purposes
num_cols = [c for c in df_census.columns if c not in ["NAME", "STATE", "COUNTY", "TRACT", "TRACT_FIPS"]] 
df_census[num_cols] = df_census[num_cols].apply(pd.to_numeric, errors="coerce")

In [14]:
df_census.dtypes

NAME                 object
TotalPop              int64
White_NonHisp         int64
Black                 int64
Asian                 int64
Hispanic              int64
MedianIncome          int64
BelowPoverty          int64
BachelorsOrHigher     int64
Male_Under18          int64
Female_Under18        int64
Male_18_34            int64
Female_18_34          int64
Male_35_64            int64
Female_35_64          int64
Male_65plus           int64
Female_65plus         int64
LaborForce            int64
Unemployed            int64
NoHealthInsurance     int64
MedianRent            int64
STATE                object
COUNTY               object
TRACT                object
TRACT_FIPS           object
dtype: object

In [15]:
# age group totals for analysis 
df_census["Age_Under18"] = df_census["Male_Under18"] + df_census["Female_Under18"]
df_census["Age_18_34"] = df_census["Male_18_34"] + df_census["Female_18_34"]
df_census["Age_35_64"] = df_census["Male_35_64"] + df_census["Female_35_64"]
df_census["Age_65plus"] = df_census["Male_65plus"] + df_census["Female_65plus"]

In [16]:
# demographic porportions 
df_census["Pct_Black"] = df_census["Black"] / df_census["TotalPop"] * 100
df_census["Pct_Hispanic"] = df_census["Hispanic"] / df_census["TotalPop"] * 100
df_census["Pct_Asian"] = df_census["Asian"] / df_census["TotalPop"] * 100
df_census["Pct_WhiteNonHisp"] = df_census["White_NonHisp"] / df_census["TotalPop"] * 100

In [17]:
# education proportions
df_census["Pct_BelowPoverty"] = df_census["BelowPoverty"] / df_census["TotalPop"] * 100
df_census["Pct_BachelorsPlus"] = df_census["BachelorsOrHigher"] / df_census["TotalPop"] * 100

#unemployment + insurance proportions
df_census["Pct_Unemployed"] = (df_census["Unemployed"] / df_census["LaborForce"]) * 100
df_census["Pct_Uninsured"] = (df_census["NoHealthInsurance"] / df_census["TotalPop"]) * 100

In [18]:
df_census.to_csv("data/processed/census_data_cleaned.csv", index=False)

### ME Data

In [19]:
df_me.head()

,Case Number,Date of Incident,Date of Death,Age,Gender,Race,Latino,Manner of Death,Primary Cause,Primary Cause Line A,...,Incident Zip Code,longitude,latitude,location,Residence City,Residence Zip,OBJECTID,Chicago Ward,Chicago Community Area,COVID Related
0,ME2025-05835,11/04/2025 02:09:00 PM,11/04/2025 01:57:00 PM,57.0,Male,Black,False,NaN,NaN,NaN,...,60621,-87.647293,41.779635,"(41.77963481, -87.64729284)",Chicago,60621,92724,16.0,ENGLEWOOD,False
1,ME2025-05834,09/24/2025 01:48:00 PM,11/04/2025 12:20:00 PM,55.0,Female,White,True,NaN,NaN,NaN,...,60609,-87.662232,41.806638,"(41.806638, -87.662232)",Chicago,60609,92725,20.0,NEW CITY,False
2,ME2025-05833,11/04/2025 12:00:00 AM,11/04/2025 12:40:00 PM,69.0,Female,White,True,NaN,NaN,NaN,...,60622,-87.696022,41.906421,"(41.906421, -87.6960225)",Chicago,60612,92720,26.0,WEST TOWN,False
3,ME2025-05832,11/04/2025 01:12:00 PM,11/04/2025 12:40:00 PM,75.0,Male,White,False,NaN,NaN,NaN,...,60622,-87.696022,41.906421,"(41.906421, -87.6960225)",Chicago,60622,92719,26.0,WEST TOWN,False
4,ME2025-05831,11/04/2025 01:41:00 PM,11/04/2025 09:48:00 AM,30.0,Male,Black,False,NaN,NaN,NaN,...,60429,-87.659707,41.581080,"(41.58108, -87.6597075)",Hazel Crest,60429,92722,NaN,NaN,False


In [20]:
df_me.dtypes

Case Number                object
Date of Incident           object
Date of Death              object
Age                       float64
Gender                     object
Race                       object
Latino                       bool
Manner of Death            object
Primary Cause              object
Primary Cause Line A       object
Primary Cause Line B       object
Primary Cause Line C       object
Secondary Cause            object
Gun Related                object
Opioid Related             object
Cold Related                 bool
Heat Related                 bool
Commissioner District     float64
Incident Address           object
Incident City              object
Incident Zip Code          object
longitude                 float64
latitude                  float64
location                   object
Residence City             object
Residence Zip              object
OBJECTID                    int64
Chicago Ward              float64
Chicago Community Area     object
COVID Related 

In [21]:
## census data is from 2019-2023, so need to filter the ME data to match that

In [47]:
df_me["Updated_Date_of_Death"] = pd.to_datetime(df_me['Date of Death'], format='%m/%d/%Y %I:%M:%S %p')

In [48]:
df_me.columns

Index(['Case Number', 'Date of Incident', 'Date of Death', 'Age', 'Gender',
       'Race', 'Latino', 'Manner of Death', 'Primary Cause',
       'Primary Cause Line A', 'Primary Cause Line B', 'Primary Cause Line C',
       'Secondary Cause', 'Gun Related', 'Opioid Related', 'Cold Related',
       'Heat Related', 'Commissioner District', 'Incident Address',
       'Incident City', 'Incident Zip Code', 'longitude', 'latitude',
       'location', 'Residence City', 'Residence Zip', 'OBJECTID',
       'Chicago Ward', 'Chicago Community Area', 'COVID Related',
       'Updated_Date_of_Death', 'Year_of_Death', 'contanins_fentanyl',
       'contains_cocaine', 'contains_heroin'],
      dtype='object')

In [49]:
df_me["Year_of_Death"] = df_me["Updated_Date_of_Death"].dt.year
df_me.head()

,Case Number,Date of Incident,Date of Death,Age,Gender,Race,Latino,Manner of Death,Primary Cause,Primary Cause Line A,...,Residence Zip,OBJECTID,Chicago Ward,Chicago Community Area,COVID Related,Updated_Date_of_Death,Year_of_Death,contanins_fentanyl,contains_cocaine,contains_heroin
3552,ME2025-02283,04/21/2023 02:08:00 PM,07/25/2023 12:00:00 AM,42.0,Male,Black,False,ACCIDENT,COMPLICATIONS OF DROWNING,COMPLICATIONS OF DROWNING,...,60707,89165,28.0,EAST GARFIELD PARK,False,2023-07-25 00:00:00,2023,False,False,False
4254,ME2025-01581,03/10/2025 12:00:00 AM,05/03/2023 11:36:00 PM,84.0,Male,Black,False,NATURAL,MALIGNANT NEOPLASM OF THE URINARY BLADDER,MALIGNANT NEOPLASM OF THE URINARY BLADDER,...,NaN,88467,NaN,NaN,False,2023-05-03 23:36:00,2023,False,False,False
4257,ME2025-01578,NaN,05/15/2023 06:29:00 AM,76.0,Male,White,False,NATURAL,HYPERTENSIVE ARTERIOSCLEROTIC CARDIOVASCULAR D...,HYPERTENSIVE ARTERIOSCLEROTIC CARDIOVASCULAR D...,...,NaN,88476,NaN,NaN,False,2023-05-15 06:29:00,2023,False,False,False
4258,ME2025-01577,03/06/2022 08:11:00 PM,03/06/2022 07:20:00 PM,71.0,Male,White,True,NATURAL,CHRONIC ETHANOLISM,CHRONIC ETHANOLISM,...,60623,88470,28.0,SOUTH LAWNDALE,False,2022-03-06 19:20:00,2022,False,False,False
12057,ME2024-00980,11/30/2023 02:23:00 PM,12/01/2023 07:52:00 AM,87.0,Female,Asian,False,NATURAL,SEPSIS. BRONCHOPNEUMONIA,SEPSIS,...,60077,80661,NaN,NaN,False,2023-12-01 07:52:00,2023,False,False,False


In [50]:
df_me = df_me[(df_me["Year_of_Death"] >= 2019) & (df_me["Year_of_Death"] <= 2023)]
df_me.head()

,Case Number,Date of Incident,Date of Death,Age,Gender,Race,Latino,Manner of Death,Primary Cause,Primary Cause Line A,...,Residence Zip,OBJECTID,Chicago Ward,Chicago Community Area,COVID Related,Updated_Date_of_Death,Year_of_Death,contanins_fentanyl,contains_cocaine,contains_heroin
3552,ME2025-02283,04/21/2023 02:08:00 PM,07/25/2023 12:00:00 AM,42.0,Male,Black,False,ACCIDENT,COMPLICATIONS OF DROWNING,COMPLICATIONS OF DROWNING,...,60707,89165,28.0,EAST GARFIELD PARK,False,2023-07-25 00:00:00,2023,False,False,False
4254,ME2025-01581,03/10/2025 12:00:00 AM,05/03/2023 11:36:00 PM,84.0,Male,Black,False,NATURAL,MALIGNANT NEOPLASM OF THE URINARY BLADDER,MALIGNANT NEOPLASM OF THE URINARY BLADDER,...,NaN,88467,NaN,NaN,False,2023-05-03 23:36:00,2023,False,False,False
4257,ME2025-01578,NaN,05/15/2023 06:29:00 AM,76.0,Male,White,False,NATURAL,HYPERTENSIVE ARTERIOSCLEROTIC CARDIOVASCULAR D...,HYPERTENSIVE ARTERIOSCLEROTIC CARDIOVASCULAR D...,...,NaN,88476,NaN,NaN,False,2023-05-15 06:29:00,2023,False,False,False
4258,ME2025-01577,03/06/2022 08:11:00 PM,03/06/2022 07:20:00 PM,71.0,Male,White,True,NATURAL,CHRONIC ETHANOLISM,CHRONIC ETHANOLISM,...,60623,88470,28.0,SOUTH LAWNDALE,False,2022-03-06 19:20:00,2022,False,False,False
12057,ME2024-00980,11/30/2023 02:23:00 PM,12/01/2023 07:52:00 AM,87.0,Female,Asian,False,NATURAL,SEPSIS. BRONCHOPNEUMONIA,SEPSIS,...,60077,80661,NaN,NaN,False,2023-12-01 07:52:00,2023,False,False,False


In [25]:
len(df_me)
# 53086 deaths between 2019 and 2023

53086

In [31]:
# create columns for deaths related to drugs 
fent_names = "FENTANYL| 4-ANILINO-N-PHENETHYLPIPERIDINE | 4-ANPP | Acetyl-alphamethyl-fentanyl | Alfentanil | Alpha-methylfentanyl | 4ANPP | 4FIBF | Alpha-methylthiofentanyl | Beta-hydroxyfentanyl | Beta-hydroxy-3 methylfentanyl | 3-methylfentanyl | 3-methylthio-fentanyl | Para-fluoro-fentanyl | Remifentanil | Sufentanil | Thiofentanyl | Carfentanil | 2-furanoylfentanyl | Furanylfentanyl | 4-anilino-N-phenethylpiperidine | 4-Fluorofentanyl | ACETYLFENTANYL | ACRYLFENTANYL | Butyrfentanyl | Lofentanil | Valerylfentanyl | Isobutyrylfentanyl"
df_me['contanins_fentanyl'] = df_me['Primary Cause'].str.contains(fent_names, case=False, na=False)

In [41]:
df_me['contains_cocaine'] = df_me['Primary Cause'].str.contains("cocaine|Benzoylmethylecgonine", case=False, na=False)
df_me["contains_heroin"] =df_me['Primary Cause'].str.contains('HEROIN', case=False, na=False)

In [43]:
df_me["Incident Zip Code"] = df_me["Incident Zip Code"].astype(str).str.extract(r"(\d{5})")
df_me["Incident Zip Code"] = df_me["Incident Zip Code"].str.zfill(5)
df_me.head()

,Case Number,Date of Incident,Date of Death,Age,Gender,Race,Latino,Manner of Death,Primary Cause,Primary Cause Line A,...,Residence Zip,OBJECTID,Chicago Ward,Chicago Community Area,COVID Related,Updated_Date_of_Death,Year_of_Death,contanins_fentanyl,contains_cocaine,contains_heroin
3552,ME2025-02283,04/21/2023 02:08:00 PM,07/25/2023 12:00:00 AM,42.0,Male,Black,False,ACCIDENT,COMPLICATIONS OF DROWNING,COMPLICATIONS OF DROWNING,...,60707,89165,28.0,EAST GARFIELD PARK,False,2023-07-25 00:00:00,2023.0,False,False,False
4254,ME2025-01581,03/10/2025 12:00:00 AM,05/03/2023 11:36:00 PM,84.0,Male,Black,False,NATURAL,MALIGNANT NEOPLASM OF THE URINARY BLADDER,MALIGNANT NEOPLASM OF THE URINARY BLADDER,...,NaN,88467,NaN,NaN,False,2023-05-03 23:36:00,2023.0,False,False,False
4257,ME2025-01578,NaN,05/15/2023 06:29:00 AM,76.0,Male,White,False,NATURAL,HYPERTENSIVE ARTERIOSCLEROTIC CARDIOVASCULAR D...,HYPERTENSIVE ARTERIOSCLEROTIC CARDIOVASCULAR D...,...,NaN,88476,NaN,NaN,False,2023-05-15 06:29:00,2023.0,False,False,False
4258,ME2025-01577,03/06/2022 08:11:00 PM,03/06/2022 07:20:00 PM,71.0,Male,White,True,NATURAL,CHRONIC ETHANOLISM,CHRONIC ETHANOLISM,...,60623,88470,28.0,SOUTH LAWNDALE,False,2022-03-06 19:20:00,2022.0,False,False,False
12057,ME2024-00980,11/30/2023 02:23:00 PM,12/01/2023 07:52:00 AM,87.0,Female,Asian,False,NATURAL,SEPSIS. BRONCHOPNEUMONIA,SEPSIS,...,60077,80661,NaN,NaN,False,2023-12-01 07:52:00,2023.0,False,False,False


In [36]:
df_me.head()

,Case Number,Date of Incident,Date of Death,Age,Gender,Race,Latino,Manner of Death,Primary Cause,Primary Cause Line A,...,Residence Zip,OBJECTID,Chicago Ward,Chicago Community Area,COVID Related,Updated_Date_of_Death,Year_of_Death,contanins_fentanyl,contains_cocaine,contains_heroin
3552,ME2025-02283,04/21/2023 02:08:00 PM,07/25/2023 12:00:00 AM,42.0,Male,Black,False,ACCIDENT,COMPLICATIONS OF DROWNING,COMPLICATIONS OF DROWNING,...,60707,89165,28.0,EAST GARFIELD PARK,False,2023-07-25 00:00:00,2023.0,False,False,False
4254,ME2025-01581,03/10/2025 12:00:00 AM,05/03/2023 11:36:00 PM,84.0,Male,Black,False,NATURAL,MALIGNANT NEOPLASM OF THE URINARY BLADDER,MALIGNANT NEOPLASM OF THE URINARY BLADDER,...,NaN,88467,NaN,NaN,False,2023-05-03 23:36:00,2023.0,False,False,False
4257,ME2025-01578,NaN,05/15/2023 06:29:00 AM,76.0,Male,White,False,NATURAL,HYPERTENSIVE ARTERIOSCLEROTIC CARDIOVASCULAR D...,HYPERTENSIVE ARTERIOSCLEROTIC CARDIOVASCULAR D...,...,NaN,88476,NaN,NaN,False,2023-05-15 06:29:00,2023.0,False,False,False
4258,ME2025-01577,03/06/2022 08:11:00 PM,03/06/2022 07:20:00 PM,71.0,Male,White,True,NATURAL,CHRONIC ETHANOLISM,CHRONIC ETHANOLISM,...,60623,88470,28.0,SOUTH LAWNDALE,False,2022-03-06 19:20:00,2022.0,False,False,False
12057,ME2024-00980,11/30/2023 02:23:00 PM,12/01/2023 07:52:00 AM,87.0,Female,Asian,False,NATURAL,SEPSIS. BRONCHOPNEUMONIA,SEPSIS,...,60077,80661,NaN,NaN,False,2023-12-01 07:52:00,2023.0,False,False,False


In [45]:
df_me.isna().sum()
# 1295 missing zip code 

Case Number                   0
Date of Incident           1693
Date of Death                 0
Age                         170
Gender                      116
Race                        231
Latino                        0
Manner of Death             252
Primary Cause                14
Primary Cause Line A         14
Primary Cause Line B      33920
Primary Cause Line C      50234
Secondary Cause           29868
Gun Related                 254
Opioid Related              254
Cold Related                  0
Heat Related                  0
Commissioner District      6914
Incident Address            822
Incident City               958
Incident Zip Code          1295
longitude                  6909
latitude                   6909
location                   6909
Residence City             1169
Residence Zip              1151
OBJECTID                      0
Chicago Ward              24105
Chicago Community Area    24105
COVID Related                 2
Updated_Date_of_Death         0
Year_of_

In [53]:
df_me = df_me.dropna(subset=["Incident Zip Code"])
len(df_me)

51791

In [54]:
df_me.isna().sum()


Case Number                   0
Date of Incident           1266
Date of Death                 0
Age                         160
Gender                      106
Race                        216
Latino                        0
Manner of Death             241
Primary Cause                14
Primary Cause Line A         14
Primary Cause Line B      33387
Primary Cause Line C      49101
Secondary Cause           29409
Gun Related                 243
Opioid Related              243
Cold Related                  0
Heat Related                  0
Commissioner District      5656
Incident Address            155
Incident City                78
Incident Zip Code             0
longitude                  5651
latitude                   5651
location                   5651
Residence City             1062
Residence Zip              1051
OBJECTID                      0
Chicago Ward              22836
Chicago Community Area    22836
COVID Related                 2
Updated_Date_of_Death         0
Year_of_

In [37]:
# AFTER CLEANING, save ME data to this!!!


# df_me = pd.read_csv('data/raw/cook_county_medical_examiner.csv')
# df_census = pd.read_csv('data/raw/census_tract_data.csv')

# # Save processed copy and compute checksum
# df_me.to_csv('data/processed/me_clean.csv', index=False)
# with open('data/processed/me_clean.csv', 'rb') as f:
#     checksum = hashlib.sha256(f.read()).hexdigest()
# print("Checksum:", checksum)